# Capella Change Detection Example: Log Ratio

In [ ]:
# Log ratio for SAR change detection

In [ ]:
# Required libraries
import numpy as np
import os
import glob
from skimage import io 
from matplotlib import pyplot as plt 
from matplotlib import rcParams
from scipy.ndimage import morphology
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance

In [ ]:
# Constants and datapath
THRSET = 3.0 # threshold setting for change detection
MORPHWINSIZE = 5
LEEFILTSIZE = 5

DATAPATH = "c:/data/mosaics/" # specify the path to data
os.chdir(DATAPATH) 

# Select the .tif files from the directory and sort the images in chronological order
image_file = np.sort(glob.glob('*.tif'))
n = len(image_file)
print("There are " + str(n) + " images to be analyzed.")

# Context image
ci = io.imread(image_file[0],0)

In [ ]:
# Lee Filter function for speckle filtering
def lee_filter(img, size):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

In [ ]:
# Main processing code block
# Ingests images two at a time, speckle filter the images, performs log ratio change detection, thresholds and saves detection map
# into an accumulator, process repeats through all image pairs and buildds a heatmap of change

# Read files in two at a time and speckle filter with a 5x5 kernel
for i in range(0,n-1):

    img1 = io.imread(image_file[i])
    lee_filt_img1 = lee_filter(img1, LEEFILTSIZE)

    img2 = io.imread(image_file[i+1])
    lee_filt_img2 = lee_filter(img2, LEEFILTSIZE)

    # Allow division by zero
    np.seterr(divide='ignore', invalid='ignore')
    
    # Calculate the log ratio of image pairs
    dIx = np.log(lee_filt_img2/lee_filt_img1)
    
    # Statistics and thresholding
    # Thresholding is empirically derived, needs manual adjustment
    thr = np.nanmean(dIx) + THRSET*np.nanstd(dIx)
    dIx[dIx < thr] = 0.0
    dIx[dIx > thr] = 1.0

    # Morphological opening to reduce false alarms    
    w = (MORPHWINSIZE, MORPHWINSIZE)
    dIx = morphology.grey_opening(dIx, size=w)
    
    # Build accumulator
    if i == 0:
        cd = dIx
    else:
        cd += dIx

In [ ]:
# Display change detection results

rcParams['figure.figsize'] = 15,8

fig, ax = plt.subplots(1,2)
ax[0].imshow(ci, 'gray');
ax[0].set_title("Context Image");
ax[1].imshow(cd);
ax[1].set_title("Change Detection Heatmap");